In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from subprocess import check_output
print(check_output(["ls", "../MNIST_data/fashion_csv/"]).decode("utf8"))

fashion-mnist_test.csv
fashion-mnist_train.csv.tar.gz



In [2]:
from keras.utils import to_categorical
import gzip
import pandas as pd
import os


# 完整的csv格式的训练集太大, 无法上传至github, 因此保存压缩文件, 在此解压缩并写入同名csv.
train_file = gzip.GzipFile("../MNIST_data/fashion_csv/fashion-mnist_train.csv.tar.gz", "r")
with open("../MNIST_data/fashion_csv/fashion-mnist_train.csv", 'wb') as f:
    f.write(train_file.read())
data_train = pd.read_csv('../MNIST_data/fashion_csv/fashion-mnist_train.csv')
data_train = data_train.drop(index=len(data_train.iloc[:, 0])-1) # 不知为什么, 最后一行全为nan, 因此删去末行.
os.remove('../MNIST_data/fashion_csv/fashion-mnist_train.csv')
data_test = pd.read_csv('../MNIST_data/fashion_csv/fashion-mnist_test.csv')

Using TensorFlow backend.


In [3]:
X_train = np.array(data_train.iloc[:, 1:])
X_test = np.array(data_test.iloc[:, 1:])
y_train = to_categorical(np.array(data_train.iloc[:, 0]))
y_test = to_categorical(np.array(data_test.iloc[:, 0]))


img_rows, img_cols = 28, 28
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D

input_shape = (img_rows, img_cols, 1)
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='adam',
          loss='categorical_crossentropy',
          metrics=['accuracy'])

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
__________

In [6]:
model.fit(X_train, y_train,
          batch_size=50,
          epochs=10,
          verbose=1,
          validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 46s 775us/step - loss: 0.5796 - acc: 0.7863 - val_loss: 0.3788 - val_acc: 0.8668
Epoch 2/10
60000/60000 [==============================] - 36s 600us/step - loss: 0.3942 - acc: 0.8564 - val_loss: 0.3023 - val_acc: 0.8890
Epoch 3/10
60000/60000 [==============================] - 34s 564us/step - loss: 0.3455 - acc: 0.8740 - val_loss: 0.2764 - val_acc: 0.8979
Epoch 4/10
60000/60000 [==============================] - 33s 558us/step - loss: 0.3178 - acc: 0.8835 - val_loss: 0.2668 - val_acc: 0.9019
Epoch 5/10
60000/60000 [==============================] - 34s 567us/step - loss: 0.3005 - acc: 0.8894 - val_loss: 0.2463 - val_acc: 0.9087
Epoch 6/10
60000/60000 [==============================] - 36s 600us/step - loss: 0.2828 - acc: 0.8947 - val_loss: 0.2387 - val_acc: 0.9128
Epoch 7/10
60000/60000 [==============================] - 36s 594us/step - loss: 0.2735 - acc: 0.8992 - val_loss: 0.

In [7]:
print('accuracy: ',score[1])
print('loss: ',score[0])

accuracy:  0.9232
loss:  0.211547868013


In [5]:
from keras.models import Sequential
model = Sequential()
help(model.evaluate)
"What's the score?"

Help on method evaluate in module keras.models:

evaluate(x, y, batch_size=32, verbose=1, sample_weight=None) method of keras.models.Sequential instance
    Computes the loss on some input data, batch by batch.
    
    # Arguments
        x: input data, as a Numpy array or list of Numpy arrays
            (if the model has multiple inputs).
        y: labels, as a Numpy array.
        batch_size: integer. Number of samples per gradient update.
        verbose: verbosity mode, 0 or 1.
        sample_weight: sample weights, as a Numpy array.
    
    # Returns
        Scalar test loss (if the model has no metrics)
        or list of scalars (if the model computes other metrics).
        The attribute `model.metrics_names` will give you
        the display labels for the scalar outputs.
    
    # Raises
        RuntimeError: if the model was never compiled.

